In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pycytominer import aggregate, annotate, normalize, feature_select, cyto_utils
from cytominer_eval import evaluate
from cytominer_eval.transform import metric_melt
from pycytominer.cyto_utils import infer_cp_features
import seaborn as sns
from sklearn.decomposition import PCA
import sys
sys.path.insert(0, "/Users/mbornhol/git/neural-profiling")
from utils import eval

In [2]:
index = pd.read_csv("../00_index.csv")
batch = pd.read_csv('/Users/mbornhol/git/neural-profiling/baseline/01_data/level_3_data/sub_level3.csv')
batch = batch[['Metadata_Batch_Number', 'Metadata_Plate', 'Metadata_Well']]
p = np.arange(0.995, 0.96, -0.005)

/Users/mbornhol/miniconda3/envs/cyto_eval/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (8,9,16,17,18,20,21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
CP = pd.read_csv('/Users/mbornhol/git/neural-profiling/baseline/01_data/level_3_data/full_level3.csv')
merge = pd.merge(CP, index, how="outer", on=["Metadata_Plate", "Metadata_Well"], indicator=True)
df_CP = CP[merge._merge == "both"]
DMSO_CP = df_CP[df_CP["Metadata_broad_sample"] == "DMSO"]
# DMSO_CP = pd.merge(DMSO_CP, batch, how="left", on=['Metadata_Plate', 'Metadata_Well'])
# DMSO_CP.columns

/Users/mbornhol/miniconda3/envs/cyto_eval/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
DMSO_CP = DMSO_CP.sample(frac=0.1)
DMSO_CP.shape

(325, 1811)

In [13]:
res = evaluate(
    profiles=DMSO_CP,
    features=infer_cp_features(DMSO_CP),
    meta_features=infer_cp_features(DMSO_CP, metadata=True),
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.999913,1.270556,0.131905
1,0.990,0.999882,1.175558,0.146101
2,0.985,0.999860,1.272283,0.021940
3,0.980,0.999839,1.200847,0.041654
4,0.975,0.999823,1.119087,0.124374
5,0.970,0.999804,1.113392,0.114130
6,0.965,0.999787,1.136991,0.057987
7,0.960,0.999770,1.193291,0.009309


In [14]:
res = evaluate(
    profiles=DMSO_CP,
    features=infer_cp_features(DMSO_CP),
    meta_features=infer_cp_features(DMSO_CP, metadata=True),
    replicate_groups=['Metadata_Plate'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.999913,6.221374,6.491462e-11
1,0.990,0.999882,5.123543,2.470925e-14
2,0.985,0.999860,5.008452,2.027357e-19
3,0.980,0.999839,5.167858,4.800106e-26
4,0.975,0.999823,4.757986,3.770664e-27
5,0.970,0.999804,4.380746,3.463840e-27
6,0.965,0.999787,4.216433,1.072733e-28
7,0.960,0.999770,3.834803,1.028550e-26


In [15]:
res = evaluate(
    profiles=DMSO_CP,
    features=infer_cp_features(DMSO_CP),
    meta_features=infer_cp_features(DMSO_CP, metadata=True),
    replicate_groups=['Metadata_Batch_Number'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.999913,3.866602,4.970575e-50
1,0.990,0.999882,2.970847,1.901248e-61
2,0.985,0.999860,2.646505,3.294856e-71
3,0.980,0.999839,2.473954,3.789559e-80
4,0.975,0.999823,2.443675,5.321199e-96
5,0.970,0.999804,2.293075,3.701100e-97
6,0.965,0.999787,2.228305,5.058740e-104
7,0.960,0.999770,2.145625,4.503232e-106


,Metadata_plate_map_name,Metadata_broad_sample,Metadata_mg_per_ml,Metadata_mmoles_per_liter,Metadata_solvent,Metadata_Plate,Metadata_Well,Metadata_pert_id,Metadata_pert_mfc_id,Metadata_pert_well,...,Metadata_target,Metadata_broad_date,Metadata_clinical_phase,Metadata_alternative_moa,Metadata_alternative_target,Metadata_Assay_Plate_Barcode,Metadata_Plate_Map_Name,Metadata_Batch_Number,Metadata_Batch_Date,Metadata_dose_recode
42649,C-7161-01-LM6-002,DMSO,0.0,0.0,DMSO,SQ00015123,B02,NaN,NaN,B02,...,NaN,NaN,NaN,NaN,NaN,SQ00015123,C-7161-01-LM6-002,2,2016-04-05,1
49664,C-7161-01-LM6-017,DMSO,0.0,0.0,DMSO,SQ00015201,F10,NaN,NaN,F10,...,NaN,NaN,NaN,NaN,NaN,SQ00015201,C-7161-01-LM6-017,1,2016-03-22,1
8959,C-7161-01-LM6-004,DMSO,0.0,0.0,DMSO,SQ00015159,F08,NaN,NaN,F08,...,NaN,NaN,NaN,NaN,NaN,SQ00015159,C-7161-01-LM6-004,2,2016-04-05,1
50328,C-7161-01-LM6-014,DMSO,0.0,0.0,DMSO,SQ00015230,B02,NaN,NaN,B02,...,NaN,NaN,NaN,NaN,NaN,SQ00015230,C-7161-01-LM6-014,3,2016-04-26,1
49379,C-7161-01-LM6-026,DMSO,0.0,0.0,DMSO,SQ00015050,J13,NaN,NaN,J13,...,NaN,NaN,NaN,NaN,NaN,SQ00015050,C-7161-01-LM6-026,4,2016-05-17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6531,C-7161-01-LM6-023,DMSO,0.0,0.0,DMSO,SQ00014819,A04,NaN,NaN,A04,...,NaN,NaN,NaN,NaN,NaN,SQ00014819,C-7161-01-LM6-023,4,2016-05-17,1
29185,C-7161-01-LM6-012,DMSO,0.0,0.0,DMSO,SQ00015223,A02,NaN,NaN,A02,...,NaN,NaN,NaN,NaN,NaN,SQ00015223,C-7161-01-LM6-012,3,2016-04-26,1
34023,C-7161-01-LM6-022,DMSO,0.0,0.0,DMSO,SQ00014813,J16,NaN,NaN,J16,...,NaN,NaN,NaN,NaN,NaN,SQ00014813,C-7161-01-LM6-022,4,2016-05-17,1
9829,C-7161-01-LM6-008,DMSO,0.0,0.0,DMSO,SQ00015105,J14,NaN,NaN,J14,...,NaN,NaN,NaN,NaN,NaN,SQ00015105,C-7161-01-LM6-008,3,2016-04-26,1


# pretrained

In [16]:
pre = pd.read_csv('/Users/mbornhol/git/neural-profiling/00_thesis/pre_trained/aggregated_efficientnet_median.csv', low_memory=False)

In [21]:
merge = pd.merge(pre, index, how="outer", on=["Metadata_Plate", "Metadata_Well"], indicator=True)
df_pre = pre[merge._merge == "both"]
DMSO_pre = df_pre[df_pre["Metadata_broad_sample"] == "DMSO"]
DMSO_pre = pd.merge(DMSO_pre, batch, how="left", on=['Metadata_Plate', 'Metadata_Well'])
features = [f for f in DMSO_pre.columns if f.startswith("eff")]
meta_features=[f for f in DMSO_pre.columns if not f.startswith("eff")]

/Users/mbornhol/miniconda3/envs/cyto_eval/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [23]:
DMSO_pre = DMSO_pre.sample(frac=0.1)
DMSO_pre.shape

(324, 6419)

In [24]:
res = evaluate(
    profiles=DMSO_pre,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.997490,1.923810,1.324484e-04
1,0.990,0.996846,1.885635,3.187611e-07
2,0.985,0.996385,2.013252,3.755325e-12
3,0.980,0.995967,2.084390,3.424125e-17
4,0.975,0.995553,1.920445,3.681090e-16
5,0.970,0.995203,1.882368,1.145166e-17
6,0.965,0.994876,1.842730,1.009919e-18
7,0.960,0.994539,1.759433,8.958986e-18


In [25]:
res = evaluate(
    profiles=DMSO_pre,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Plate'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.997490,53.788561,4.190055e-156
1,0.990,0.996846,40.843500,9.101552e-208
2,0.985,0.996385,33.853703,2.095067e-231
3,0.980,0.995967,32.695994,1.622761e-268
4,0.975,0.995553,29.241796,1.294840e-275
5,0.970,0.995203,27.620228,4.070710e-288
6,0.965,0.994876,24.542062,5.948539e-280
7,0.960,0.994539,23.344602,1.018954e-284


In [26]:
res = evaluate(
    profiles=DMSO_pre,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Batch_Number'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.997490,520.737429,0.0
1,0.990,0.996846,532.983115,0.0
2,0.985,0.996385,271.353905,0.0
3,0.980,0.995967,211.168330,0.0
4,0.975,0.995553,121.481883,0.0
5,0.970,0.995203,90.445923,0.0
6,0.965,0.994876,69.970525,0.0
7,0.960,0.994539,52.010797,0.0


,Unnamed: 0,Metadata_Aggregate_On,Metadata_Well_Position,Metadata_Plate,Metadata_Well,Metadata_broad_sample,Metadata_moa,Metadata_mmoles_per_liter,Metadata_dose_recode,Metadata_Concentration,Metadata_Treatment_ID,Metadata_Compound,Metadata_pert_iname,Metadata_Treatment_Replicate,Metadata_Treatment,Metadata_Plate_Map_Name,Metadata_Split,Metadata_Model
0,0,well,SQ00014812/A01,SQ00014812,A01,DMSO,unknown,0.0,0,NaN,0,DMSO,NaN,1,DMSO@NA,C-7161-01-LM6-022,Training,efficientnet
1,1,well,SQ00014812/A02,SQ00014812,A02,DMSO,unknown,0.0,0,NaN,0,DMSO,NaN,2,DMSO@NA,C-7161-01-LM6-022,Training,efficientnet
2,2,well,SQ00014812/A03,SQ00014812,A03,DMSO,unknown,0.0,0,NaN,0,DMSO,NaN,3,DMSO@NA,C-7161-01-LM6-022,Training,efficientnet
3,3,well,SQ00014812/A04,SQ00014812,A04,DMSO,unknown,0.0,0,NaN,0,DMSO,NaN,4,DMSO@NA,C-7161-01-LM6-022,Training,efficientnet
4,4,well,SQ00014812/A05,SQ00014812,A05,DMSO,unknown,0.0,0,NaN,0,DMSO,NaN,5,DMSO@NA,C-7161-01-LM6-022,Training,efficientnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3234,3234,well,SQ00015233/J14,SQ00015233,J14,DMSO,unknown,0.0,0,NaN,0,DMSO,NaN,3248,DMSO@NA,C-7161-01-LM6-014,Training,efficientnet
3235,3235,well,SQ00015233/J15,SQ00015233,J15,DMSO,unknown,0.0,0,NaN,0,DMSO,NaN,3249,DMSO@NA,C-7161-01-LM6-014,Training,efficientnet
3236,3236,well,SQ00015233/J16,SQ00015233,J16,DMSO,unknown,0.0,0,NaN,0,DMSO,NaN,3250,DMSO@NA,C-7161-01-LM6-014,Training,efficientnet
3237,3237,well,SQ00015233/J17,SQ00015233,J17,DMSO,unknown,0.0,0,NaN,0,DMSO,NaN,3251,DMSO@NA,C-7161-01-LM6-014,Training,efficientnet


# Trained

In [27]:
train = pd.read_csv('/Users/mbornhol/git/neural-profiling/training/runs/1028/1028_aggregated_median.csv', low_memory=False)

In [28]:
merge = pd.merge(train, index, how="outer", on=["Metadata_Plate", "Metadata_Well"], indicator=True)
df_train = train[merge._merge == "both"]
DMSO_train = df_train[df_train["Metadata_broad_sample"] == "DMSO"]
DMSO_train = pd.merge(DMSO_train, batch, how="left", on=['Metadata_Plate', 'Metadata_Well'])
features = [f for f in DMSO_train.columns if f.startswith("eff")]
meta_features=[f for f in DMSO_train.columns if not f.startswith("eff")]

In [29]:
DMSO_train = DMSO_train.sample(frac=0.1)
DMSO_train.shape

(325, 1299)

In [30]:
res = evaluate(
    profiles=DMSO_train,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.696456,0.869444,7.594149e-01
1,0.990,0.565618,1.146253,1.881984e-01
2,0.985,0.481442,1.208633,5.814364e-02
3,0.980,0.419968,1.510655,9.625040e-06
4,0.975,0.369380,1.560057,1.069363e-07
5,0.970,0.326959,1.510575,9.791781e-08
6,0.965,0.296176,1.621470,1.048439e-11
7,0.960,0.270663,1.565370,2.895119e-11


In [31]:
res = evaluate(
    profiles=DMSO_train,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Plate'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.696456,192.018832,0.0
1,0.990,0.565618,130.776328,0.0
2,0.985,0.481442,105.956083,0.0
3,0.980,0.419968,84.994115,0.0
4,0.975,0.369380,79.996459,0.0
5,0.970,0.326959,73.020437,0.0
6,0.965,0.296176,72.319485,0.0
7,0.960,0.270663,64.384570,0.0


In [32]:
res = evaluate(
    profiles=DMSO_train,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Batch_Number'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.696456,inf,0.0
1,0.990,0.565618,inf,0.0
2,0.985,0.481442,1131.581284,0.0
3,0.980,0.419968,354.181130,0.0
4,0.975,0.369380,182.241053,0.0
5,0.970,0.326959,106.671063,0.0
6,0.965,0.296176,70.216893,0.0
7,0.960,0.270663,49.556401,0.0


In [22]:
# compare augmented to non augmented

non = pd.read_csv('/Users/mbornhol/git/neural-profiling/training/runs/1008/1008_aggregated_median.csv')
# aug = pd.read_csv('/Users/mbornhol/git/neural-profiling/training/runs/1010/1010_aggregated_median.csv')

/Users/mbornhol/miniconda3/envs/cyto_eval/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
# merge = pd.merge(train, index, how="outer", on=["Metadata_Plate", "Metadata_Well"], indicator=True)
# df_train = train[merge._merge == "both"]
DMSO_train = non[non["Metadata_broad_sample"] == "DMSO"]
DMSO_train = pd.merge(DMSO_train, batch, how="left", on=['Metadata_Plate', 'Metadata_Well'])
features = [f for f in DMSO_train.columns if f.startswith("eff")]
meta_features=[f for f in DMSO_train.columns if not f.startswith("eff")]

In [24]:
DMSO_train = DMSO_train.sample(frac=0.2)
DMSO_train.shape

(650, 1299)

In [25]:
res = evaluate(
    profiles=DMSO_train,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.816401,1.234509,2.136448e-02
1,0.990,0.736719,1.446433,5.407601e-08
2,0.985,0.671016,1.478828,2.144463e-12
3,0.980,0.618052,1.395927,1.183150e-11
4,0.975,0.572537,1.351588,1.348435e-11
5,0.970,0.530035,1.326719,5.621123e-12
6,0.965,0.493150,1.335183,3.507317e-14
7,0.960,0.459520,1.355011,1.888766e-17


In [26]:
res = evaluate(
    profiles=DMSO_train,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Plate'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.816401,205.575377,0.0
1,0.990,0.736719,120.766909,0.0
2,0.985,0.671016,97.613952,0.0
3,0.980,0.618052,85.128542,0.0
4,0.975,0.572537,79.875692,0.0
5,0.970,0.530035,76.245798,0.0
6,0.965,0.493150,74.854989,0.0
7,0.960,0.459520,71.898063,0.0


In [27]:
res = evaluate(
    profiles=DMSO_train,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Batch_Number'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.816401,inf,0.0
1,0.990,0.736719,1760.565692,0.0
2,0.985,0.671016,561.773918,0.0
3,0.980,0.618052,268.900670,0.0
4,0.975,0.572537,170.767753,0.0
5,0.970,0.530035,123.532721,0.0
6,0.965,0.493150,94.433929,0.0
7,0.960,0.459520,76.114370,0.0


In [28]:
aug = pd.read_csv('/Users/mbornhol/git/neural-profiling/training/runs/1010/1010_aggregated_median.csv')

/Users/mbornhol/miniconda3/envs/cyto_eval/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
# merge = pd.merge(train, index, how="outer", on=["Metadata_Plate", "Metadata_Well"], indicator=True)
# df_train = train[merge._merge == "both"]
DMSO_train = aug[aug["Metadata_broad_sample"] == "DMSO"]
DMSO_train = pd.merge(DMSO_train, batch, how="left", on=['Metadata_Plate', 'Metadata_Well'])
features = [f for f in DMSO_train.columns if f.startswith("eff")]
meta_features=[f for f in DMSO_train.columns if not f.startswith("eff")]

In [30]:
DMSO_train = DMSO_train.sample(frac=0.2)
DMSO_train.shape

(650, 1299)

In [31]:
res = evaluate(
    profiles=DMSO_train,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Well'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.814019,1.330701,2.532495e-03
1,0.990,0.743888,1.371228,4.648273e-06
2,0.985,0.691756,1.522666,4.610313e-14
3,0.980,0.646553,1.485794,5.449343e-16
4,0.975,0.609611,1.469680,2.406548e-18
5,0.970,0.575118,1.451150,4.299967e-20
6,0.965,0.544724,1.450213,6.884852e-23
7,0.960,0.516841,1.433432,4.259189e-24


In [32]:
res = evaluate(
    profiles=DMSO_train,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Plate'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.814019,132.314612,0.0
1,0.990,0.743888,78.961853,0.0
2,0.985,0.691756,64.713009,0.0
3,0.980,0.646553,56.769989,0.0
4,0.975,0.609611,50.584607,0.0
5,0.970,0.575118,46.099973,0.0
6,0.965,0.544724,44.200794,0.0
7,0.960,0.516841,43.147660,0.0


In [33]:
res = evaluate(
    profiles=DMSO_train,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Batch_Number'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.814019,2157.539910,0.0
1,0.990,0.743888,417.908198,0.0
2,0.985,0.691756,239.305704,0.0
3,0.980,0.646553,141.263801,0.0
4,0.975,0.609611,92.157361,0.0
5,0.970,0.575118,68.540549,0.0
6,0.965,0.544724,55.037679,0.0
7,0.960,0.516841,46.610475,0.0


In [34]:
sphere = normalize(
    profiles=non,
    features=features,
    samples="Metadata_broad_sample == 'DMSO'",
    method='spherize',
    output_file='none'
)

In [43]:
DMSO_train = sphere[sphere["Metadata_broad_sample"] == "DMSO"]
DMSO_train = pd.merge(DMSO_train, batch, how="left", on=['Metadata_Plate', 'Metadata_Well'])
DMSO_train = DMSO_train.sample(frac=0.2)
DMSO_train.shape
features = [f for f in DMSO_train.columns if f.startswith("eff")]
meta_features=[f for f in DMSO_train.columns if not f.startswith("eff")]

In [44]:
res = evaluate(
    profiles=DMSO_train,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Plate'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.069264,134.323122,0.0
1,0.990,0.051536,64.683636,0.0
2,0.985,0.045189,43.909474,0.0
3,0.980,0.041189,33.987174,0.0
4,0.975,0.038360,27.232235,0.0
5,0.970,0.036224,22.599750,0.0
6,0.965,0.034486,19.285379,0.0
7,0.960,0.033013,17.105015,0.0


In [45]:
res = evaluate(
    profiles=DMSO_train,
    features=features,
    meta_features=meta_features,
    replicate_groups=['Metadata_Batch_Number'],
    operation="enrichment",
    similarity_metric="pearson",
    enrichment_percentile=p,
)
res

,enrichment_percentile,threshold,ods_ratio,p-value
0,0.995,0.069264,61.928278,0.0
1,0.990,0.051536,13.238270,0.0
2,0.985,0.045189,7.460675,0.0
3,0.980,0.041189,5.357751,0.0
4,0.975,0.038360,4.385099,0.0
5,0.970,0.036224,3.775039,0.0
6,0.965,0.034486,3.320240,0.0
7,0.960,0.033013,3.025243,0.0


KeyError: "['Unnamed: 0'] not in index"